In [1]:
import sys
#sys.path.append("/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection")
sys.path.append("/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2")
#sys.path.append("/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/training")
sys.argv = ["train.py"]

In [4]:
from training.detectors import DETECTOR

ModuleNotFoundError: No module named 'tensorboard'

## Imports

In [6]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pytorch_grad_cam import XGradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

### Load Xception Model

In [ ]:
import yaml
from training.detectors import xception_detector
#i needed to append my analysis path to system to access the bcos saved models

def load_config(path, additional_args = {}):
    # parse options and load config
    with open(path, 'r') as f:
        config = yaml.safe_load(f)
    try:# KAI: added this, to ensure it finds the config file
        #with open('/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/training/config/train_config.yaml', 'r') as f:
        with open('/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/training/config/train_config.yaml', 'r') as f:
            config2 = yaml.safe_load(f)
    except FileNotFoundError:
        #with open(os.path.expanduser('/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/training/config/train_config.yaml'), 'r') as f:
        with open(os.path.expanduser('/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/training/config/train_config.yaml'), 'r') as f:
            config2 = yaml.safe_load(f)
    if 'label_dict' in config:
        config2['label_dict']=config['label_dict']
    config.update(config2)
    # config['local_rank']=args.local_rank
    if config['dry_run']:
        config['nEpochs'] = 0
        config['save_feat']=False
    for key, value in additional_args.items():
        config[key] = value
    return config

#path = "/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/training/config/detector/xception.yaml"
path = "/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/training/config/detector/xception.yaml"
additional_args = {'test_batchSize': 12, 
                   'pretrained':
                    #f'/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/weights/resnet/ckpt_best.pth'
                    f'/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/weights/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-10-17-17/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-12-19-31/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-17-23-41/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-21-26-03/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-21-43-33/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-08-09-13-00/test/avg/ckpt_best.pth'
                  }
config = load_config(path, additional_args=additional_args)
print("Registered models:", DETECTOR.data.keys())
model_class = DETECTOR[config['model_name']]
model = model_class(config)

model.eval()

### Laoding in grid 

In [ ]:
class MyToTensor(transforms.ToTensor):
    def __init__(self):
        """
        Same as transforms.ToTensor, except that if input to __call__ is already tensor, the input is returned unchanged
        """
        super().__init__()

    def __call__(self, input_img):
        if not isinstance(input_img, torch.Tensor):
            return super().__call__(input_img)
        return input_img
    
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize if necessary
    MyToTensor(),            # Converts image to tensor if not already          
])


class CustomImageDataset(Dataset):
    def __init__(self, folder_paths, transform=None):
        """
        Args:
            folder_paths (dict): Dictionary where keys are folder paths, and values are the labels.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.image_files = []  # Store (image_path, label) tuples
        for fp, label in folder_paths.items():
            # List all subdirectories within the folder
            subfolders = [os.path.join(fp, d) for d in os.listdir(fp) if os.path.isdir(os.path.join(fp, d))]
        # Include images from the root directory as well as subfolders
        self.image_files.extend(
            [(os.path.join(fp, f), label) for f in os.listdir(fp) if f.endswith((".png", ".jpg"))]
        )

        for folder_path in subfolders:
            self.image_files.extend(
                [(os.path.join(folder_path, f), label) for f in os.listdir(folder_path) if f.endswith((".png", ".jpg"))]
            )
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path, label = self.image_files[idx]  # Get image path and its label
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label, img_path


#file_path_deepfakebench = {"/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/datasets/GPG_grids/3ch": 1}
file_path_deepfakebench = {"/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/datasets/2x2_images": 1}

dataset = CustomImageDataset(file_path_deepfakebench, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for images, label, img_path in dataloader:
    print(f"Batch of images shape: {images.shape}")
    break

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for img_batch, label_batch, path_batch in dataloader:
    print(f"Batch of images shape: {img_batch.shape}")

    # Iterate through the first 5 images in the batch
    num_images = min(len(img_batch), 20)
    for i in range(num_images):
        img = img_batch[i].unsqueeze(0).to(device)  # Process a single image
        label = label_batch[i]
        img_path = path_batch[i]
        img = img.requires_grad_(True)
        
        data_dict = {'image': img, 'label': label}
        
        model.zero_grad()
        out = model(data_dict)
        
        prob = out['prob']
        print('predicted prob', prob)


## X Grad Cam